In [21]:
import requests
import pandas as pd
import numpy as np
import math
import plotly.graph_objs as go
from plotly.grid_objs import Grid, Column
import plotly.plotly as py

In [2]:
reports_r = requests.get('https://peridash.ml/api/reports')
measurements_r = requests.get('https://peridash.ml/api/measurements')
epsas_r = requests.get('https://peridash.ml/api/epsas')
variables_r = requests.get('https://peridash.ml/api/variables')
indicators_r = requests.get('https://peridash.ml/api/indicators')

In [3]:
reports_df = pd.read_json(reports_r.text)
measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)

complete_reports_df = pd.merge(reports_df, epsas_df, left_on='epsa', right_on='url')
complete_measurements_df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

rdf = complete_reports_df
mdf = complete_measurements_df
vdf = pd.read_json(variables_r.text)
idf = pd.read_json(indicators_r.text)

In [4]:
ind_names = [
    'Rendimiento actual de la fuente', 'Uso eficiente del recurso',
    'Cobertura de muestras de agua potable',
    'Conformidad de los análisis de agua potable realizados',
    'Dotación', 'Continuidad por racionamiento', 'Continuidad por corte',
    'Cobertura del servicio de agua potable',
    'Cobertura del servicio de alcantarillado sanitario',
    'Cobertura de micromedición',
    'Incidencia extracción de agua cruda subterránea ',
    'Índice de tratamiento de agua residual', 'Control de agua residual',
    'Capacidad instalada de planta de tratamiento de agua potable',
    'Capacidad instalada de planta de tratamiento de agua residual ',
    'Presión del servicio de agua potable',
    'Índice de agua no contabilizada en producción',
    'Índice de agua no contabilizada en la red',
    'Densidad de fallas en tuberías de agua potable',
    'Densidad de fallas en conexiones de agua potable',
    'Densidad de fallas en tuberías de agua residual',
    'Densidad de fallas en conexiones de agua residual',
    'Índice de operación eficiente', 'Prueba ácida',
    'Eficiencia de recaudación', 'Índice de endeudamiento total', 'Tarifa media',
    'Costo unitario de operación', 'Índice de ejecución de inversiones',
    'Personal calificado', 'Número de empleados por cada 1000 conexiones',
    'Atención de reclamos'
]
ind_units = ['%', '%', '%', '%', 'l/hab/día', 'hr/día', '%', '%', '%', '%',
             '%', '%', '%', '%', '%', '%', '%', '%', 'fallas/100km',
             'fallas/1000conex.', 'fallas/100km', 'fallas/1000conex.', '%', '-',
             '%', '%', '%CUO(Bs.)', '%TM(Bs.)', '%', '%', 'empleados/1000conex.', '%']

colors = [
    '#1f77b4','#ff7f0e','#2ca02c',
    '#d62728', '#9467bd', '#8c564b',
    '#e377c2', '#7f7f7f', '#bcbd22','#17becf'
]

def get_ind_name(ind):
    return ind_names[int(ind[3:])-1]

def get_ind_unit(ind):
    return ind_units[int(ind[3:])-1]

In [5]:
years = list(rdf.year.sort_values().unique()) # [2014, 2015, 2016, 2017]
categories = list(rdf.category.sort_values().unique()) # ['A', 'B', 'C', 'D']

plane_dims = ['v22', 'v23']
size_dim = 'v22'

xi = int(plane_dims[0][1:]) # 17
yi = int(plane_dims[1][1:]) # 18

xname = vdf[vdf.var_id==xi].name.iloc[0] # 'Número total de conexiones de agua potable activas medidas y no medidas'
yname = vdf[vdf.var_id==yi].name.iloc[0] # 'Número total de conexiones de alcantarillado sanitario activas '
xunit = vdf[vdf.var_id==xi].unit.iloc[0] # 'conex.'
yunit = vdf[vdf.var_id==yi].unit.iloc[0] # 'conex.'

code_to_cat = {code: cat for code,cat in zip(epsas_df.code, epsas_df.category)}

In [12]:
cat_to_y = {cat: len(categories) - i for i,cat in enumerate(categories)}

columns = []

for year in years:
    for category in categories:
        frdf = rdf[(rdf.year == year) & (rdf.category == category)]
                       
        percentages = [x/y * 100 for x,y in zip(frdf.v23, frdf.v22)]
        
        column_name1 = f'{year}_{category}_text'
        column_data1 = [f'{code}<br>{"%.2f"%(p)}%<br>Población Abastecida: {a}<br>Población Total: {b}' for code,a,b,p in zip(frdf.code, frdf.v23, frdf.v22,percentages)]
        column1 = Column(column_data1, column_name1)
        
        column_name2 = f'{year}_{category}_x'
        column_data2 = percentages
        column2 = Column(column_data2, column_name2)
        
        column_name3 = f'{year}_{category}_y'
        column_data3 = [cat_to_y[code_to_cat[code]] for code in frdf.code]
        column3 = Column(column_data3, column_name3)
        
        columns += [column1, column2, column3]

grid = Grid(columns)
py.grid_ops.upload(grid, 'population_2017')

'https://plot.ly/~sergiochumacero/84/'

In [13]:
def create_trace(year, category):
    return dict(
        xsrc= grid.get_column_reference(f'{year}_{category}_x'),
        ysrc= grid.get_column_reference(f'{year}_{category}_y'),
        marker= dict(
            symbol='line-ns',
            size=25,
            opacity=0.7,
            line = dict(
              color = colors[dict(A=0, B=1, C=2, D=3)[category]],
              width = 2
            )
#             size=[0.0 if math.isnan(x) else x for x in grid_data[f'{year}_{category}_{size_dim}']],
#             sizemode='area',
#             sizeref=1000,
        ),
        mode= 'markers',
        textsrc= grid.get_column_reference(f'{year}_{category}_text'),
        name= 'Categoría: ' + category,
        hoverinfo = 'text',
    )

base_data = [create_trace(years[0], category) for category in categories]

In [14]:
def create_frame(year):
    frame_data = [create_trace(year, category) for category in categories]
    return dict(data=frame_data, name=str(year))

frames = [create_frame(year) for year in years]

In [22]:
animation_settings = dict(
    frame = dict(duration=1200, redraw=False),
    fromcurrent = False,
    transition = dict(duration=1200, easing='cubic-in-out'),
)

def make_step(year):
    return dict(
        method = 'animate',  
        args = [[year], animation_settings],
        label= year
    )
steps = [make_step(str(year)) for year in years] 

sliders = [dict(
    active = 1,
    currentvalue = {
        'prefix': 'Año: ',
        'font': {'size': 20},
        'visible': True,
        'xanchor': 'right'
    },
    steps = steps,
    yanchor= 'top',
    xanchor= 'left',
    pad= {'b': 10, 't': 50},
    len= 0.9,
    x= 0.1,
    y= 0,
)]

updatemenus = [dict(
    buttons= [dict(
        args= [[str(y) for y in years], animation_settings],
        label= 'Animar',
        method= 'animate',
    )],
    direction= 'left',
    pad= dict(r=10, t=87),
    showactive= False,
    type= 'buttons',
    x= 0.1,
    y= 0,
    xanchor= 'right',
    yanchor= 'top',
)]

layout = go.Layout(
    title='Población Abastecida',
    hovermode='y',
    width=1000,
    legend=dict(x=.1, y=1.1, orientation='h'),
    xaxis=dict(
        title='Porcentaje (%): Población Abastecida / Población Total del Área de Servicio * 100',
        range=[35,100],
        autorange=False,
        tickmode='linear',
        tick0 = 35,
        dtick = 5,
    ),
    yaxis=dict(visible=False),
#     xaxis={'title':f'Variable {str(xi)}: {xname} ({xunit})'},
#     yaxis={'title':f'Variable {str(yi)}: {yname} ({yunit})'},
    plot_bgcolor='#dfe8f3',
    sliders= sliders,
    updatemenus = updatemenus,
)

In [23]:
figure = go.Figure(data=base_data, frames=frames, layout=layout)

In [26]:
py.icreate_animations(figure, filename='populations_2017')

PlotlyRequestError: Hi there, you've reached the threshold of 100 combined image exports and chart saves per 24h period. If you need to raise your daily limit, please consider upgrading to a paid plan
(see: https://plot.ly/products/cloud ).